In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image

Intializing grid which stores pixel values at different latitudes and longitudes generated from image matrix of many images

In [ ]:
import numpy as np
grid = np.zeros((50000, 10000,3),dtype=np.uint8)

directory contains path for images to be stitched 

tif_path contains path for the tmc file of the images to be stiched (to access latitudes and longitudes)

In [ ]:
directory='Enter path'  
tif_path='Enter path'

In [ ]:
#return longitude and latitude of a point
def open(tif_file,x,y):
   with rasterio.open(tif_file) as src:
        band1 = src.read(1)               #band1 stores the single band data of .tiff file
        height = band1.shape[0]
        width = band1.shape[1]
        cols, rows = np.meshgrid(x,y)
        xs, ys = rasterio.transform.xy(src.transform, rows, cols)
        lons= np.array(xs)
        lats = np.array(ys)
   return lons,lats   

grid_lons,grid_lats denote upperleft longitude and lattitude of tmc image

In [ ]:
grid_lons,grid_lats = open(tif_path,0,0)

Assuming image name contains pixel rows and columns value Ex. 0_0.png,0_256.png,256_256.png,256_512.png,etc
(images are of size 256*256)

In [ ]:
# to get coordinates(row,col) of upper left corner from filename of sliced pngs
def get_coordinate(file):
  items = file.split("/")
  items = items[-1].split(".")
  items = items[0].split("_")
  return int(items[0]),int(items[1])

In [ ]:
upscale_factor=16

In [ ]:
lons_per_pixel,_=(open(tif_path,1,0)-open(tif_path,0,0))/upscale_factor
_,lats_per_pixel=(open(tif_path,0,1)-open(tif_path,0,0))/upscale_factor

In [ ]:

for filename in os.listdir(directory):
    f = os.path.join(directory, filename)

    x,y= get_coordinate(f)
    lons,lats=open(tif_path,x,y)
  
    row=int((lons-grid_lons)*lons_per_pixel)    # to get row_index and col_index of a image in final grid
    col=int((lats-grid_lats)*lats_per_pixel)

    img = Image.open(f)
    pixel_matrix = np.asarray(img)
    
    grid_row=row
    grid_col=col

    for row in pixel_matrix:
      grid_col=col
      for pixel in row:
        grid[grid_row,grid_col]=pixel         # assigning pixel value in grid
        grid_col=grid_col+1
      grid_row=grid_row+1

In above function, for overlapping images we can take median values but due to high computational cost, we are taking last image value.

saving final image

In [ ]:
img = Image.fromarray(grid)
img.save('stitched.png')